### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_47797/2486307642.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?  \n\nBecause it wanted to be a polyunsaturated!'}

In [20]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "Kannada"})

{'input': 'a parrot',
 'language': 'Kannada',
 'text': 'ನಾಯಿ: "ನೀನು ಏನು ಹೇಳುತ್ತೀಯ?"\n\nಕೋಕ್ಕನಿಗೆ: "ನಾನು ಪರ್ವಾಳ ಬಿಜೆ ಎನ್ನಿಸಿಕೊಂಡಿರುವೆ!"\n\nನಾಯಿ: "ಮಾಡಿಕೊಳ್ಳಿ, ನಾನು ನಿಮ್ಮನ್ನು ಕೇಳುತ್ತಿದ್ದೇನೆ!"\n\n(ಗಾತಿ/ಆಡಿಯೋ: ನಾನು ಯಾರಿಗೆ ಕೇಳಿಸುತ್ತೇನೆ?)'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [15]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to Hindi: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="hindi_translation"
)

In [17]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "hindi_translation"],
)

overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})

{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': '**Review of Pizza Salami Experience**\n\nI recently ordered a Pizza Salami, and I must say, it was an incredibly disappointing experience. From start to finish, it was nothing short of awful.\n\nFirst off, the delivery was delayed by over 30 minutes, which is frustrating in itself. When the pizza finally arrived, it was lukewarm and lacked the fresh-out-of-the-oven quality that one hopes for when ordering pizza. \n\nOpening the box was my first letdown – the pizza was poorly assembled, with toppings sliding off the sides. The crust was chewy and devoid of flavor, leaving me longing for the crispy, golden edges I usually enjoy. As for the salami, it taste bland and greasy, lacking the richness and kick that makes it a go-to topping. \n\nMoreover, the sauce was overly sweet and didn’t complement the other ingredients at all. I found myself picking off the toppings just to salvage what little enjoyment I could from 

### Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [21]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [22]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    # Here we are creating a dictionary of chains, where the key is the name of the chain and the value is the chain itself.
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x118b6e710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x118b30d90>, root_client=<openai.OpenAI object at 0x118b88610>, root_async_client=<openai.AsyncOpenAI object at 0x118b59690>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'neutral': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring 

In [24]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [30]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # RouterOutputParser is used to parse the output of the router chain into a dictionary
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was not good!"})



> Entering new MultiPromptChain chain...
negative: {'input': 'I ordered Pizza Salami for $9.99 and it was not good!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for $9.99 and it was not good!',
 'text': 'Here are the potential negative aspects highlighted in the text:\n\n1. **Quality of Food**: The phrase "it was not good" suggests dissatisfaction with the taste or quality of the Pizza Salami, indicating that it did not meet expectations.\n\n2. **Value for Money**: At a price point of $9.99, the buyer may feel that they did not receive good value for their money, as the quality did not align with the cost.\n\n3. **Expectation vs. Reality**: There might have been a discrepancy between what the customer expected from the Pizza Salami and what was actually delivered, leading to disappointment.\n\n4. **Potential Health Concerns**: If the pizza was of low quality, it could raise concerns about the ingredients used, suggesting possible health risks or lack of freshness.\n\n5. **Wasted Money**: The dissatisfaction implies that the customer feels their expenditure was unwise, possibly leading to regret about the purch